In [77]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [11]:
gdf = gpd.read_file("/mnt/store-lidarhd/projet-LHD/IA/BDForet/Data/PureForestID/metadata/PureForestID-patches.gpkg")
gdf = gdf[gdf.split.isin(["test","val"])]
gdf = gdf[["split", "patch_num", "bdforet_identifier","bdforet_cat_code", "bdforet_cat_name","inventory_species_label", ]]
print(gdf.columns)
gdf.head()

Index(['split', 'patch_num', 'bdforet_identifier', 'bdforet_cat_code',
       'bdforet_cat_name', 'inventory_species_label'],
      dtype='object')


,split,patch_num,bdforet_identifier,bdforet_cat_code,bdforet_cat_name,inventory_species_label
0,val,00035_00000,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
1,val,00035_00001,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
2,val,00035_00002,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
3,val,00035_00003,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
4,val,00035_00004,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14


In [94]:
PREDICTIONS_FOLDER = Path("/mnt/store-lidarhd/projet-LHD/IA/MYRIA3D-SHARED-WORKSPACE/CGaydon/runs/2023-11-03/17-47-28")
PREDICTIONS_FILE = PREDICTIONS_FOLDER / "predictions.csv"
df = pd.read_csv(PREDICTIONS_FILE)
df.head(n=10) # duplicates !
df = df.drop_duplicates("patch_id",keep="first")

mapping = {0: "FF1-09-09_Hêtre",
  1: "FF1-10-10_Châtaignier",
  2: "FF1-14-14_Robinier",
  3: "FF1G01-01_Chêne_décidus",
  4: "FF1G06-06_Chêne_sempervirent",
  5: "FF2-51-51_Pin_maritime",
  6: "FF2-52-52_Pin_sylvestre",
  7: "FF2-57-57_Pin_alep",
  8: "FF2-63-63-Mélèze",
  9: "FF2-64-64_Douglas",
  10: "FF2G53-53_Pin_laricio_Pin_noir",
  11: "FF2G61-61_Sapin-Epicéa"}
df["targets"] = df["targets"].apply(lambda t: mapping[t])
df["preds"] = df["preds"].apply(lambda t: mapping[t])
df.head()

,patch_id,preds,targets
0,00194_00000,FF1-10-10_Châtaignier,FF2G61-61_Sapin-Epicéa
3,00266_00000,FF2G61-61_Sapin-Epicéa,FF2G61-61_Sapin-Epicéa
6,00266_00001,FF2G61-61_Sapin-Epicéa,FF2G61-61_Sapin-Epicéa
9,00266_00002,FF2G61-61_Sapin-Epicéa,FF2G61-61_Sapin-Epicéa
12,00266_00003,FF2G61-61_Sapin-Epicéa,FF2G61-61_Sapin-Epicéa


In [95]:
merge = gdf.merge(df, left_on="patch_num", right_on="patch_id", how="inner")
# merge["OK"] = (merge["preds"] == merge["targets"]).astype(int)
# merge["NOK"] = (merge["preds"] != merge["targets"]).astype(int)
print(len(merge))
# merge.head()

19463


In [96]:
groups = merge.groupby(["bdforet_identifier","bdforet_cat_code","targets","preds"]).size().reset_index()
pivot = groups.pivot(index=["bdforet_identifier","targets"],columns="preds",values=0).fillna(0).astype(int)
pivot["consensus"] = pivot.idxmax(axis=1)
pivot = pivot.reset_index()
pivot["accurate"] = pivot["consensus"] == pivot["targets"]
pivot.to_csv(PREDICTIONS_FOLDER/"predictions-pivot-by-polygon.csv")
pivot

preds,bdforet_identifier,targets,FF1-09-09_Hêtre,FF1-10-10_Châtaignier,FF1-14-14_Robinier,FF1G01-01_Chêne_décidus,FF1G06-06_Chêne_sempervirent,FF2-52-52_Pin_sylvestre,FF2-57-57_Pin_alep,FF2-63-63-Mélèze,FF2-64-64_Douglas,FF2G53-53_Pin_laricio_Pin_noir,FF2G61-61_Sapin-Epicéa,consensus,accurate
0,FORESTIE0000000000100093,FF1-14-14_Robinier,0,0,12,0,0,0,0,0,0,0,0,FF1-14-14_Robinier,True
1,FORESTIE0000000000100104,FF1-14-14_Robinier,0,0,36,0,0,0,0,0,0,0,0,FF1-14-14_Robinier,True
2,FORESTIE0000000000100240,FF1-14-14_Robinier,0,0,9,0,0,0,0,0,0,0,0,FF1-14-14_Robinier,True
3,FORESTIE0000000000401455,FF1G01-01_Chêne_décidus,0,0,49,83,27,6,0,0,0,0,0,FF1G01-01_Chêne_décidus,True
4,FORESTIE0000000000402323,FF1G01-01_Chêne_décidus,145,40,0,628,0,2,0,0,0,0,0,FF1G01-01_Chêne_décidus,True
5,FORESTIE0000000000402638,FF1G01-01_Chêne_décidus,4,47,0,358,0,0,2,0,0,0,0,FF1G01-01_Chêne_décidus,True
6,FORESTIE0000000000407464,FF2-52-52_Pin_sylvestre,0,0,0,0,6,12,1,0,0,0,0,FF2-52-52_Pin_sylvestre,True
7,FORESTIE0000000000522889,FF1-09-09_Hêtre,395,0,65,1,0,0,0,15,0,0,0,FF1-09-09_Hêtre,True
8,FORESTIE0000000000522924,FF1-09-09_Hêtre,21,0,0,3,0,0,0,3,0,0,0,FF1-09-09_Hêtre,True
9,FORESTIE0000000000526662,FF2-63-63-Mélèze,3,0,0,2,0,3,0,536,0,0,0,FF2-63-63-Mélèze,True


In [97]:
synthesis = pivot.groupby("targets").agg({"accurate":"mean","bdforet_identifier":"size"}).round(2)
synthesis = synthesis.rename(columns={"accurate":"accuracy","bdforet_identifier":"num_bdforet_polygons"})
synthesis.to_csv(PREDICTIONS_FOLDER/"predictions-accuracy-by-class.csv")
synthesis


preds,accuracy,num_bdforet_polygons
targets,,
FF1-09-09_Hêtre,1.00,3
FF1-10-10_Châtaignier,0.50,2
FF1-14-14_Robinier,0.92,12
FF1G01-01_Chêne_décidus,0.57,7
FF1G06-06_Chêne_sempervirent,0.67,3
FF2-52-52_Pin_sylvestre,0.75,4
FF2-57-57_Pin_alep,0.50,2
FF2-63-63-Mélèze,1.00,1
FF2-64-64_Douglas,0.40,5


In [105]:
# Lien entre erreur et taille de polygone -> pas vraiment.
pivot[~pivot.accurate][pivot.columns[2:13]].sum(axis=1)

11      32
17       3
18      14
19     200
22      54
23       3
25      22
27     186
37    4362
40     256
41     301
42      71
dtype: int64